# Extended solution

This time let's extend baseline with some heuristics

0. First of all, we will try to split compound sentences into few smaller ones and apply coreference resolution on the chunks. 
1. We will use compound BERT embeddings for sentences as before
2. But in addition let's make some morphological tree manupulations
    - Some parts of speech contain less useful information than the others
3. As well as calculate special weights for some words
    - Named entities intuitively seem more important that common nouns
